## This is the major start of our code, where we are getting data from github repository

In [ ]:
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install
! git clone https://github.com/neheller/kits19.git

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 53 not upgraded.
Need to get 6,877 kB of archives.
After this operation, 16.4 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.11.0 [6,877 kB]
Fetched 6,877 kB in 1s (13.4 MB/s)
debconf: unable to init

In [ ]:
%cd kits19
!pip3 install -r /content/kits19/requirements.txt
! python -m starter_code.get_imaging

/content/kits19
300 cases to download...
Download 1/300: 
case_00000: 100% 225959/225960 [00:04<00:00, 55735.16KB/s]
Download 2/300: 
case_00001: 100% 276387/276388 [00:04<00:00, 56264.35KB/s]
Download 3/300: 
case_00002: 100% 101967/101968 [00:01<00:00, 52339.29KB/s]
Download 4/300: 
case_00003: 100% 118681/118682 [00:02<00:00, 51294.15KB/s]
Download 5/300: 
case_00004: 100% 25269/25270 [00:00<00:00, 36380.65KB/s]
Download 6/300: 
case_00005: 100% 313477/313478 [00:05<00:00, 57029.71KB/s]
Download 7/300: 
case_00006: 100% 78010/78011 [00:01<00:00, 47687.39KB/s]
Download 8/300: 
case_00007: 100% 25826/25827 [00:00<00:00, 37506.94KB/s]
Download 9/300: 
case_00008: 100% 107068/107069 [00:02<00:00, 52981.67KB/s]
Download 10/300: 
case_00009: 100% 34134/34135 [00:00<00:00, 35732.92KB/s]
Download 11/300: 
case_00010: 100% 23541/23542 [00:00<00:00, 35046.14KB/s]
Download 12/300: 
case_00011: 100% 41268/41269 [00:00<00:00, 43260.05KB/s]
Download 13/300: 
case_00012: 100% 43185/43186 [00:01<00

In [ ]:
%cd /content/kits19/

/content/kits19


In [ ]:
!pip install medicaltorch

## To visualize our data we need to use this Block

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,8))
# from medicaltorch import datasets as mt_datasets
# from skimage import color
# pair = mt_datasets.SegmentationPair2D(input_filename, gt_filename) 
# slice_pair = pair.get_pair_slice(175)
# input_slice = slice_pair["input"]
# gt_slice = slice_pair["gt"]

# img = input_slice
# label = gt_slice
# plt.imshow(label, cmap = 'gray')
# plt.show()


In [ ]:
# import numpy as np
# np.unique(label)  #ile wyróżnia nam klas

## Loading Libraries for performing segmentation using Deep Learning Techniques

In [ ]:
import torch
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler, Adam, SGD
from torch.utils.data import Dataset, DataLoader
from medicaltorch import datasets as mt_datasets
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from medicaltorch import models as mt_models
from keras.utils import to_categorical as cat
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Using TensorFlow backend.


device(type='cuda')

## Reading the data paths for processing and getting cases numbers in file_name variable

In [ ]:
import os
train_path = "/content/kits19/data/"
cases =  next(os.walk(train_path))
file_name = cases[1]

## Splitting the data into training and Validation

In [ ]:
train_data = file_name[:240]
validation_data = file_name[240:]

## getting the model from pre build library

In [ ]:
model = mt_models.Unet(drop_rate=0.2, bn_momentum=0.8)
for param in model.parameters():
    param.requires_grad = True 

In [ ]:
def transform(img, lbl):
  
  img = img.astype(np.float32)
  lbl = lbl.astype(np.float32)
  img -= np.mean(img)
  img /= 255.0
    
  return img, lbl

## Declaring classes to make their dataloaders

In [ ]:

class TrainDataset(Dataset):
    def __init__(self, df):
        self.df = df
        # self.transform = transform   
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        global img
        global lbl
        cases = train_data[idx]
        file_path = train_path + cases +'/'
        # print(file_path)
        content = next(os.walk(file_path))
        if 'segmentation.nii.gz' not in content[2]:
          pass
        else:
          input_filename = file_path + content[2][-1]
          gt_filename = file_path + content[2][0]
          pair = mt_datasets.SegmentationPair2D(input_filename, gt_filename)
          slice_pair = pair.get_pair_slice(175)
          img = slice_pair["input"]
          label = slice_pair["gt"]
          img, lbl = transform(img, label)
          img = np.resize(img, (224, 224))  
          img = torch.from_numpy(img).unsqueeze_(0)
          # print('Img shape:', img.shape)
          lbl = np.resize(lbl, (224, 224))  
          lbl = torch.from_numpy(lbl).unsqueeze_(0)
          # print('Img shape:', img.shape)
        return img, lbl 


In [ ]:

class ValidDataset(Dataset):
    def __init__(self, df):
        self.df = df
        # self.transform = transform
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        global img
        global lbl
        cases = validation_data[idx]
        file_path = train_path + cases +'/'
        content = next(os.walk(file_path))
        if 'segmentation.nii.gz' not in content[2]:
          pass
        else:
          input_filename = file_path + content[2][-1]
          gt_filename = file_path + content[2][0]
          pair = mt_datasets.SegmentationPair2D(input_filename, gt_filename)
          slice_pair = pair.get_pair_slice(175)
          img = slice_pair["input"]
          label = slice_pair["gt"]
          img, lbl = transform(img, label)
          img = np.resize(img, (224, 224))  
          img = torch.from_numpy(img).unsqueeze_(0)
          
          lbl = np.resize(lbl, (224, 224))  
          lbl = torch.from_numpy(lbl).unsqueeze_(0)
          # print('Img shape:', img.shape)
        return img, lbl 
  

## form the dataloaders for further processing of data... Here i have data and label images both...

In [ ]:
train_dataset = TrainDataset(train_data) 
train_loader = DataLoader(train_dataset, batch_size=10, num_workers=0)

valid_dataset = ValidDataset(validation_data) 
valid_loader = DataLoader(valid_dataset, batch_size=10, num_workers=0)

## Loss function and changing optimizers here

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Dice Loss Class

In [ ]:
import torch
from torch.autograd import Function


class DiceCoeff(Function):
    """Dice coeff for individual examples"""

    def forward(self, input, target):
        self.save_for_backward(input, target)
        eps = 0.0001
        self.inter = torch.dot(input.view(-1), target.view(-1))
        self.union = torch.sum(input) + torch.sum(target) + eps

        t = (2 * self.inter.float() + eps) / self.union.float()
        return t

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):

        input, target = self.saved_variables
        grad_input = grad_target = None

        if self.needs_input_grad[0]:
            grad_input = grad_output * 2 * (target * self.union - self.inter) \
                         / (self.union * self.union)
        if self.needs_input_grad[1]:
            grad_target = None

        return grad_input, grad_target


def dice_coeff(input, target):
    """Dice coeff for batches"""
    if input.is_cuda:
        s = torch.FloatTensor(1).cuda().zero_()
    else:
        s = torch.FloatTensor(1).zero_()

    for i, c in enumerate(zip(input, target)):
        s = s + DiceCoeff().forward(c[0], c[1])

    return s / (i + 1)

# So here our all training starts...

In [ ]:
num_epochs = 5
model.to(device)
history = []

for epoch in range(num_epochs): 
    train_loss_total = 0.0 
    valid_loss_total = 0.0 
    tot = 0
    tot_train = 0
    print('Epoch has started') 
      
    for images, labels in iter(train_loader):
      data = images.to(torch.float)
      data = images.to(device)
      label = labels.to(torch.float)
      label = labels.to(device) 
      # print('step 1 check')
      outputs = model(data)
      # print('blablabla')
      loss = criterion(outputs, label) 
      optimizer.zero_grad()
      loss.backward() 
      # print('step 2 check')
      
      optimizer.step() 
      pred = (outputs > 0.5).float()
      tot_train += dice_coeff(pred, label).item()
      train_loss_total  += loss.item()

    for data, label in iter(valid_loader): 
      
      data = data.to(torch.float)
      data = data.to(device)
      label = label.to(torch.float)
      label = label.to(device) 
      outputs = model(data)
      loss = criterion(outputs, label)
      pred = (outputs > 0.5).float()
      tot += dice_coeff(pred, label).item()
      valid_loss_total  += loss.item()
 
    train_loss_total_avg = train_loss_total / len(train_loader)
    valid_loss_total_avg = valid_loss_total / len(valid_loader)
    dice_train = tot_train/len(train_loader)
    dice_valid = tot/len(valid_loader)

    history.append([train_loss_total_avg, valid_loss_total_avg, dice_train, dice_valid]) 
    print('epoch number ', epoch, "		",'Training loss value', train_loss_total_avg , "		", 'Validation Loss Value', valid_loss_total_avg  \
          , "		", 'Dice Loss for train is', dice_train, "  ", 'Dice loss for validation is', dice_valid)
    
history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_dice_loss', 'valid_dice_loss'])  

Epoch has started


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch number  0 		 Training loss value 163.01694520314535 		 Validation Loss Value 140.3655128479004 		 Dice Loss for train is 0.5288949906826019    Dice loss for validation is 0.34937089184919995
Epoch has started
epoch number  1 		 Training loss value 116.8987242380778 		 Validation Loss Value 107.93188858032227 		 Dice Loss for train is 0.3025410504390796    Dice loss for validation is 0.31768765300512314
Epoch has started


## Getting curves for both losses...

In [ ]:

plt.figure(figsize=(8, 6))
for c in ['train_loss', 'valid_loss']:
    plt.plot(history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Negative Log Likelihood')
plt.title('Training and Validation Losses')

plt.figure(figsize=(8, 6))
for c in ['train_dice_loss', 'valid_dice_loss']:
    plt.plot(100 * history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Accuracy')
plt.title('Training and Validation Accuracy')

In [ ]:
pyplot.subplot(1, 3, 1) # have two plots in 1 row two columns, first plot
# assuming im is batch x channel x h x w and channel is RGB
im, mask = next(iter(valid_loader))
pred = model(im)
pyplot.plot(im[0].detach().cpu().permute(1, 3, 0))
pyplot.subplot(1, 3, 2) # second plot
pyplot.plot(mask[0].detach().cpu())
pyplot.subplot(1, 3, 3) # third plot
pyplot.plot(pred[0].detach().cpu())